In [ ]:
import pandas as pd

from lhorizon.handlers import query_all_lhorizons, construct_lhorizon_list

problem: what will the approximate sun angle be at an unimportant spot on Mars over a period of several years, at five-minute intervals?

In [ ]:
# defining position, time span, and so on

# what is the body center of the observing location?
mars_horizons_id = '499'

# what is the topocentric location relative to that body center?
observer_latitude = 18.4
# note that Horizons always treats east-longitude as negative for Mars and 
# other prograde bodies *except* the Earth, Moon, and Sun
observer_longitude = -77.5
observer_elevation = 0

# what is the observer looking at?
# (this can also target topocentric coordinates on the target body using the same syntax,
# but obviously this is not important in this particular application.)
sun_horizons_id = '10'

# start time, stop time, and observation interval
start = '2020-01-01T00:00:00'
stop = '2023-02-01T00:00:00'
# this can be 's', 'm', 'h', 'd', 'y' for fixed time intervals;
# alternatively, just an integer will give a number of intervals
# equal to (start time - stop_time) / step.
step = "5m"

# the only quantities we care about are apparent altitude and azimuth. Horizons will need
# to return less data, and the query will go faster, if we ask only for these. see below for a full
# list of observer-table quantity codes.
quantities = "4"

In [ ]:
# make a bunch of horizons requests at palatable-to-horizons sizes.
lhorizons = construct_lhorizon_list(
    target=sun_horizons_id,
    origin={
        'lon': observer_longitude,
        'lat': observer_latitude,
        'elevation': observer_elevation,
        'body': mars_horizons_id
    },
    epochs = {
        'start': start,
        'stop': stop,
        'step': step
    },
    query_options={'quantities': quantities},
    chunksize=85000
)

In [ ]:
%%time
# this is only about 8 MB of data, but it requires extensive backend calculations to generate
# and Horizons is sometimes bandwidth-limited. Depending on your connection and how Horizons 
# is feeling on an individual day, it could take between 30 seconds and 15 minutes to retrieve it all.
query_all_lhorizons(lhorizons)

In [ ]:
# process all these data as CSV and concatenate them into a single pandas dataframe
sun_positions = pd.concat(
    [lhorizon.table() for lhorizon in lhorizons]
)
# horizons bulk queries don't work well with the 'only when object is visible'
# flags, so we handle it here
solar_angle_table = sun_positions.loc[sun_positions['alt'] > 0].reset_index(drop=True)
solar_angle_table

In [ ]:
# write out to csv
solar_angle_table.to_csv(
    "solar_angle_table " + start + " to " + stop + ".csv",
    index=None
)

In [ ]:
# footnote: OBSERVER table quantity codes

#   1. Astrometric RA & DEC  16. Sub Sun Pos. Ang & Dis *31. Obs eclip. lon & lat
#  *2. Apparent RA & DEC     17. N. Pole Pos. Ang & Dis  32. North pole RA & DEC
#   3.   Rates; RA & DEC     18. Helio eclip. lon & lat  33. Galactic latitude
#  *4. Apparent AZ & EL      19. Helio range & rng rate  34. Local app. SOLAR time
#   5.   Rates; AZ & EL      20. Obsrv range & rng rate  35. Earth -> site lt-time
#   6. Sat. X & Y, pos. ang  21. One-Way Light-Time     >36. RA & DEC uncertainty
#   7. Local app. sid. time  22. Speed wrt Sun & obsrvr >37. POS error ellipse
#   8. Airmass               23. Sun-Obs-Targ ELONG ang >38. POS uncertainty (RSS)
#   9. Vis mag. & surf brt.  24. Sun-Targ-Obs~PHASE ang >39. Range & rng-rate sig.
#  10. Illuminated fraction  25. Targ-Obsrv-Moon/Illum% >40. Doppler/delay sigmas
#  11. Defect of illumin.    26. Obs-Primary-Targ angle  41. True anomaly angle
#  12. Sat. angle separ/vis  27. Radial & -vel posn.ang *42. Local app. hour angle
#  14. Obs sub-lon & sub-lat 29. Constellation name      43. PHASE angle & bisect
#  15. Sun sub-lon & sub-lat 30. Delta-T (TDB - UT)